In [ ]:
#@test {"skip": true}

!which python
import nest_asyncio
nest_asyncio.apply()

%load_ext tensorboard

from matplotlib import pyplot as plt
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:

import collections

from IPython.display import display, HTML, IFrame

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

print(tff.__version__)

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

np.random.seed(0)
#def greetings():
#  display(HTML('<b><font size="6" color="#ff00f4">Greetings, virtual tutorial participants!</font></b>'))
#  return True
#l = tff.federated_computation(greetings)()


# User clustering

In the previous tutorials, we learned how to set up model and data pipelines, and use these to perform federated training using the `tff.learning` API.

Of course, this is only the tip of the iceberg when it comes to FL research. In this tutorial, we are going to discuss how to implement federated learning algorithms *without* deferring to the `tff.learning` API. We aim to accomplish the following:

**Goals:**


*   Understand the general structure of federated learning algorithms.
*   Explore the *Federated Core* of TFF.
*   Use the Federated Core to implement Federated Averaging directly.




## Preparing the input data
We first load and preprocess the EMNIST dataset included in TFF. We essentially use the same code as in the first tutorial.

In [226]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [227]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 28,28,1]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [228]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

In [229]:
#Choose 10 clients randomly
client_ids = np.random.choice(emnist_train.client_ids, size=NUM_CLIENTS, replace=False)

federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## Preparing the model

We use the same model as the first tutorial, which has a single hidden layer, followed by a softmax layer.

In [230]:
def create_keras_model():
  data_format = 'channels_last'
  initializer = tf.keras.initializers.RandomNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(28, 28,1)),
      tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
      tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
      tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
      tf.keras.layers.Dropout(rate=0.75),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu', kernel_initializer=initializer),
      tf.keras.layers.Dropout(rate=0.5, seed=1),
      tf.keras.layers.Dense(62, kernel_initializer=initializer),
      tf.keras.layers.Softmax()
  ])

We wrap this Keras model as a `tff.learning.Model`.

In [231]:
dummy_model = model_fn()
tf_dataset_type = tff.SequenceType(dummy_model.input_spec)
model_weights_type = server_init.type_signature.result
str(tf_dataset_type), str(model_weights_type)
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

In [272]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [233]:
@tf.function
def client_update(model, dataset,server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.weights.trainable
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

In [234]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.weights.trainable
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

In [235]:
@tff.tf_computation()
def server_init():
  model = model_fn()
  return model.weights.trainable

In [236]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

In [237]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

In [238]:
@tff.federated_computation
def initialize_fn():
   #return tff.federated_map(server_update_fn, model_weights)
   return tff.federated_value(server_init(), tff.SERVER)

Remember the 4 elements of an FL algorithm?

1. A server-to-client broadcast step.
2. A local client update step.
3. A client-to-server upload step.
4. A server update step.

Now that we've built up the above, each part can be compactly represented as a single line of TFF code. This simplicity is why we had to take extra care to specify things such as federated types!

In [239]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights, client_weights

In [240]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

In [241]:
str(federated_algorithm.initialize.type_signature),str(federated_algorithm.next.type_signature)

('( -> <float32[3,3,1,32],float32[32],float32[3,3,32,64],float32[64],float32[9216,128],float32[128],float32[128,62],float32[62]>@SERVER)',
 '(<server_weights=<float32[3,3,1,32],float32[32],float32[3,3,32,64],float32[64],float32[9216,128],float32[128],float32[128,62],float32[62]>@SERVER,federated_dataset={<x=float32[?,28,28,1],y=int32[?,1]>*}@CLIENTS> -> <<float32[3,3,1,32],float32[32],float32[3,3,32,64],float32[64],float32[9216,128],float32[128],float32[128,62],float32[62]>@SERVER,{<float32[3,3,1,32],float32[32],float32[3,3,32,64],float32[64],float32[9216,128],float32[128],float32[128,62],float32[62]>}@CLIENTS>)')

In [242]:
state=federated_algorithm.initialize()

In [265]:
fed_avg = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.03),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0, momentum=0.9))
    

In [244]:
NUM_ROUNDS=10
with summary_writer.as_default():
  for round_num in range(0, NUM_ROUNDS):
    state,client_weights = federated_algorithm.next(state, federated_train_data)

In [ ]:
print(client_weights[0])
# It is a list of 8 tensors , each tensor is a layer weight

In [260]:
NUM_OF_CLUSTERS = 4
print(client_ids)

['f3416_28' 'f2028_18' 'f3118_09' 'f1086_15' 'f2454_73' 'f0981_17'
 'f3344_26' 'f0928_42' 'f1779_01' 'f0800_00']


In [261]:
#Randomly choose two client weights for initialization
q = np.random.choice([i for i in range(0,8)], size=NUM_OF_CLUSTERS, replace=False)
print(q)
q_ids = client_ids[q]
print(q_ids)

[5 6 4 3]
['f0981_17' 'f3344_26' 'f2454_73' 'f1086_15']


In [262]:
@tf.function
def client_forward_pass(model, dataset, cluster_weights ):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.weights.trainable
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, cluster_weights)

  # Use the client_optimizer to update the local model.
  total_loss=0.0
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)
      total_loss=total_loss+outputs.loss

  return total_loss

In [ ]:
#clustering code here
NUM_CLIENTS =20
model = model_fn()

#select train client ids
import random
shuffled_ids = emnist_train.client_ids.copy()
random.shuffle(shuffled_ids)
clients = shuffled_ids[0:600]

#clients = emnist_train.client_ids.copy()

NUM_ROUNDS_SGD = 5

clusters = {client_ids[idx]: [{'weights':client_weights[idx]},client_ids[idx]] for idx in q}

while len(clients)!=0:
    
    # Randomly pick p clients
    print('Number of clients left '+ str(len(clients)))
          
    if len(clients) < NUM_CLIENTS :
         p = np.random.choice(clients, size=len(clients), replace=False)
    else :
         p = np.random.choice(clients, size=NUM_CLIENTS, replace=False)
            
    #subtrack the selected clients from k 
    for j in p:
        clients.remove(j)
        
    print('Number of clients left '+ str(len(clients)))
    
    #for each client in p
    for x in p:
        
        # create dataset for client x
        train_data = preprocess(emnist_train.create_tf_dataset_for_client(x))
        
        #Initialize loss 
        loss = []
        
        #looping over k clusters
        for key in list(clusters.keys()):
            # calculate the loss for every client in p with clusters
            loss.append(client_forward_pass(model, train_data, clusters[key][0]['weights']))
        
        #assign client_id to the cluster with minimum loss
        min_loss = min(loss)
        min_loss_index = loss.index(min_loss)
        l = list(clusters.keys())
        clusters[l[min_loss_index]] = clusters[l[min_loss_index]] + [x]
    
    # Run SGD for few steps
    for cluster_id in list(clusters.keys()):
        
        sample_clients = clusters[cluster_id][1:]
        # Size of the cluster
        print('Size of cluster '+str(cluster_id)+'is'+ str(len(sample_clients)))
        
        
        #Prepare dataset for all clients in the cluster
        train_data = make_federated_data(emnist_train, sample_clients)
        # Use all the clients in the cluster and initialize with the cluster model

        weights = clusters[cluster_id][0]['weights']
        state = fed_avg.initialize()
        model = model_fn()
        model_weights = model.weights.trainable
  
        tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, weights)
        
        state = tff.learning.state_with_new_model_weights(state,
                    trainable_weights=[v.numpy() for v in model.weights.trainable],
                    non_trainable_weights=[v.numpy() for v in model.weights.non_trainable] )


        # Use local datasets of the clients and run few steps of SGD
        for round_num in range(0,NUM_ROUNDS_SGD):
            state, metrics = fed_avg.next(state, train_data)
        print('metrics={}'.format(metrics['train']))
        # Perform Fedavg using models from all the clients with mk (number of samples with each client)
        # Clients send mk*hk(local weights of the clients)
        # Global_cluster_model = Fedavg(m1h1,m2h2...mkhk)
        clusters[cluster_id][0]['weights'] = state.model.trainable
    
    
    
    # Get the weights

## Evaluating the algorithm

In [ ]:
clusters

In [270]:
# Construct federated evaluation computation here:
evaluation = tff.learning.build_federated_evaluation(model_fn)

In [276]:
for cluster_id in list(clusters.keys()):
    sample_clients = clusters[cluster_id][1:]
    federated_test_data = make_federated_data(emnist_test, sample_clients)
    state = fed_avg.initialize()
    weights = clusters[cluster_id][0]['weights']
    model = model_fn()
    model_weights = model.weights.trainable
  
    tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, weights)
        
    state = tff.learning.state_with_new_model_weights(state,
                    trainable_weights=[v.numpy() for v in model.weights.trainable],
                    non_trainable_weights=[v.numpy() for v in model.weights.non_trainable] )
    print('Iam here')
    test_metrics = evaluation(state.model, federated_test_data)
    print(test_metrics)

Iam here
OrderedDict([('sparse_categorical_accuracy', 0.9539049), ('loss', 0.16496158)])
Iam here
OrderedDict([('sparse_categorical_accuracy', 0.9421285), ('loss', 0.20896013)])
Iam here
OrderedDict([('sparse_categorical_accuracy', 0.925776), ('loss', 0.21819805)])
Iam here
OrderedDict([('sparse_categorical_accuracy', 0.94848484), ('loss', 0.17035863)])


In [ ]:
# Run evaluation on the test data here, using the federated model produced from 
# training:
